# Anomaly

This section describes the use of anomaly generators within the `payment_simulator` Package, which facilitates the simulation of transaction anomalies in Real-Time Gross Settlement (RTGS) systems. It works by simulating anomalies that vary in probability and intensity over a defined period. This is useful for testing the system's response to increasing levels of disruption.


In [1]:
import numpy as np
from payment_simulator.anomaly import AbstractAnomalyGenerator, AnomalyGenerator

np.random.seed(1234)

## Anomaly Generator

The `AnomalyGenerator` class, derived from `AbstractAnomalyGenerator`, is designed to simulate transaction anomalies, specifically mimicking bank run scenarios where there's a sudden increase in transactions within a specific time frame. This class adjusts probabilities and exponential distribution parameters gradually according to a pre-set rate. It calculates both the likelihood and the size of anomalies based on the current period, smoothly transitioning these values from their starting to ending points over the anomaly period. This approach helps in creating realistic anomalies in financial transaction simulations.

Here we can see the `AnomalyGenerator` in action. Anomalies are only generated between the pre-defined period, which is from period 3 to 7.

In [2]:
anom_gen = AnomalyGenerator(
    anomaly_start=3,
    anomaly_end=7,
    prob_start=0.4,
    prob_end=0.8,
    lambda_start=1.5,
    lambda_end=2.5,
    rate=0.5,
)

for i in range(10):
    anomaly = anom_gen(i)
    print(f"{i:2d} {anomaly:3.3f}")

 0 0.000
 1 0.000
 2 0.000
 3 0.000
 4 0.000
 5 1.534
 6 2.951
 7 2.059
 8 0.000
 9 0.000


## Custom Generator

You can create your own anomaly generator by extending the `AbstractAnomalyGenerator` class. To do this, you need to define how anomalies are calculated by implementing the `__call__` method. We've provided two examples of such custom generators:

1. `UniformAnomalyGenerator`: This generator produces values within a specified range between two numbers.
2. `RateAnomalyGenerator`: This generator linearly increases values based on a predetermined rate.

Both implementations involve calculating a probability of anomalies occurrence. This allows for flexible and stochastic modeling of anomalies in your simulations.

In [3]:
class UniformAnomalyGenerator(AbstractAnomalyGenerator):
    def __init__(self, min: float = 0, max: float = 1, prob: float = 0.5) -> None:
        self.min = min
        self.max = max
        self.prob = prob
        
    def __call__(self, current_period: int = 0) -> float:
        return np.random.binomial(1, self.prob) * np.random.uniform(self.min, self.max)

In [4]:
uanom_gen = UniformAnomalyGenerator(0, 10, 0.4)

for i in range(10):
    print(f"{i} {uanom_gen(i):.2f}")

0 0.00
1 0.00
2 6.51
3 0.00
4 0.00
5 4.36
6 1.44
7 7.05
8 0.00
9 0.00


In [5]:
class RateAnomalyGenerator(AbstractAnomalyGenerator):
    def __init__(self, rate: float = 1, prob: float = 0.5) -> None:
        self.rate = rate
        self.prob = prob
        
    def __call__(self, x, current_period: int = 0) -> float:
        return np.random.binomial(1, self.prob) * x * self.rate

In [6]:
ranom_anom = RateAnomalyGenerator(1.3)

for i in range(10):
    print(f"{i} {10+i} {(10+i) * .3:.2f} {ranom_anom(10 + i, i):.2f}")

0 10 3.00 0.00
1 11 3.30 0.00
2 12 3.60 0.00
3 13 3.90 16.90
4 14 4.20 18.20
5 15 4.50 19.50
6 16 4.80 0.00
7 17 5.10 22.10
8 18 5.40 0.00
9 19 5.70 24.70
